In [ ]:
import os
from ipywidgets import Dropdown, VBox
from IPython.display import display

import rasterio
import numpy as np
import matplotlib.pyplot as plt

def show_tiff_rgb(path, rgb_bands=(4, 3, 2), stretch=True):
    """
    Display a quick RGB preview of a multiband GeoTIFF.

    Parameters
    ----------
    path : str
        Path to the GeoTIFF.
    rgb_bands : tuple
        1-based band numbers (e.g., Sentinel-2: 4,3,2).
    stretch : bool
        Apply contrast stretching to improve visibility.
    """
    with rasterio.open(path) as src:
        # rasterio is 1-based indexing for bands
        rgb = np.stack([src.read(b) for b in rgb_bands], axis=0)

        # Simple min-max stretch
        if stretch:
            rgb = rgb.astype(float)
            for i in range(3):
                low, high = np.percentile(rgb[i], (2, 98))
                rgb[i] = np.clip((rgb[i] - low) / (high - low + 1e-6), 0, 1)

        # Convert HWC for matplotlib
        rgb = np.moveaxis(rgb, 0, -1)

    plt.figure(figsize=(8, 8))
    plt.imshow(rgb)
    plt.axis("off")
    plt.title(f"RGB preview: {path}")


def find_tifs(root="../data"):
    tif_files = []
    for base, dirs, files in os.walk(root):
        for f in files:
            if f.lower().endswith((".tif", ".tiff")):
                tif_files.append(os.path.join(base, f))
    return sorted(tif_files)

# Search for TIFFs
tif_list = find_tifs("../data")

if not tif_list:
    print("No TIFF files found under data/")
else:
    tif_dropdown = Dropdown(
        options=tif_list,
        description="Select TIFF:",
        layout={"width": "600px"}
    )

    print("Select a TIFF from the dropdown:")
    display(VBox([tif_dropdown]))

In [ ]:
selected_tif = tif_dropdown.value
print("Selected:", selected_tif)

In [ ]:
show_tiff_rgb(selected_tif)

In [ ]:

def on_change(change):
    if change["name"] == "value":
        plt.close("all")
        print("Selected:", change["new"])
        show_tiff_rgb(change["new"])

tif_dropdown.observe(on_change)
